In [ ]:
import numpy as np
import pandas as pd
import json

In [ ]:
# Open and read the JSON file
file_path = 'data/rf5_synthetic_ballots (2).json'  # Replace with your actual file path

with open(file_path, 'r') as file:
    data = json.load(file)  # Load the JSON data into a variable

In [ ]:
### Calculate Budget allocation

budget_allocation =[]

# Collect budgets based on category assignment
for entry in data:
    budget = entry["budget"]
    budget_allocation.append(budget)

budget_median = np.median(budget_allocation)

budget_median

7225000.0

In [ ]:
### Calculate Category allocation

# 1. Isolate the category budget votes: Each badgeholder will have voted on how to allocate OP to all categories (e.g. [Category1: 33%; Category2: 33%; Category3: 34%])
# 2. Calculate the median of Category allocation
# 3. Adjust category allocations to match 100%

category_scores = {
    "eth_core": [],
    "op_rnd": [],
    "op_tooling": []
}

for entry in data:
    allocations = entry["category_allocations"]
    for category, score in allocations.items():
        category_scores[category].append(score)

# Calculate the median for each category
category_medians = {category: np.median(scores) for category, scores in category_scores.items()}

# Normalize the category medians
total_median = sum(category_medians.values())
normalized_category_medians = {category: (median / total_median) * 100 for category, median in category_medians.items()}

# Output the normalized category allocations
for category, score in normalized_category_medians.items():
    print(f"{category}: {score}")

eth_core: 31.349772227147117
op_rnd: 35.236806951170976
op_tooling: 33.413420821681896


In [ ]:
### Calculate Project Scores

# 1. Isolate the Project votes: Each badgeholder will have voted by submitting percentages reflecting the allocation of OP to the projects within a category (e.g. [Project1: 10; Project2: 4;….]).
# 2. Remove Project votes with a COI: For each badgeholder, they will have a number of `null` votes. These should not be considered for results calculation
# 3. Calculate the median scores of projects
# 4. Adjust scores to match 100%
# 5. Repeat step 1 - 4 for all 3 categories

# Calculate median project allocation under each category
project_allocations_by_category = {
    "eth_core": {},
    "op_tooling": {},
    "op_rnd": {}
}

# Collect project allocations based on category assignment
for entry in data:
    category = entry["category_assignment"]
    project_allocations = entry["project_allocations"]

    for project_id, allocation in project_allocations.items():
        if allocation is not None:  # Exclude None values
            if project_id not in project_allocations_by_category[category]:
                project_allocations_by_category[category][project_id] = []
            project_allocations_by_category[category][project_id].append(allocation)

# Calculate median project allocation for each project under each category
median_project_allocations = {}
for category, projects in project_allocations_by_category.items():
    median_project_allocations[category] = {}
    for project_id, allocations in projects.items():
        median_project_allocations[category][project_id] = np.median(allocations)

normalized_project_scores = {}

for category, scores in median_project_allocations.items():
    total_score = sum(scores.values())
    normalized_project_scores[category] = {project_id: (score / total_score) * 100 for project_id, score in scores.items()}


# Display median results
# print("\nMedian Project Allocations:")
# for category, projects in median_project_allocations.items():
#     print(f"{category}:")
#     for project_id, median_value in projects.items():
#         print(f"  Project {project_id}: {median_value}")

# Output the normalized project scores
for category, scores in normalized_project_scores.items():
    print(f"\n{category}:")
    print(f"no. of prjects: {len(scores)}")
    for project_id, normalized_score in scores.items():
        print(f"Project ID: {project_id}, Normalized Score: {normalized_score:.2f}%")


eth_core:
no. of prjects: 46
Project ID: 0x82ef80ecf81b0bab897fe9ab595544db44b14708993f91c5f5f457f869d2af1d, Normalized Score: 2.83%
Project ID: 0xbdc75c7f2d9d4f797174656b47ef920cc968b82a1235efec4a37816fc8077ef9, Normalized Score: 1.79%
Project ID: 0xdda0798a5eb762366cd5aa5de87f042567946b47f5cf1d776077a2b00d75c575, Normalized Score: 2.26%
Project ID: 0xd7f9140cd24d5332f81c9a12d5d7da80f0c61f69db015073727064f0b975c875, Normalized Score: 2.20%
Project ID: 0xf615a763fa6599d40e52ee2e15ec77f208324c5863a157cdeb243639936476e8, Normalized Score: 1.67%
Project ID: 0x73deb2a75792f7c688dc87a3080ce18637b7d241ad2b551272fb21aee72d6bf7, Normalized Score: 1.18%
Project ID: 0x762c363ac739a057eb4d414fd17916a2b098a00c7ddeae32d021f6df572561e1, Normalized Score: 1.96%
Project ID: 0x56cb9a566631cf918d682a40877cb4ffe7dfc4d3cbec30f5cc9a1a71c8083618, Normalized Score: 1.84%
Project ID: 0x0af93b34c4c047485dca950b6a1f063727ea08ea6363988322a45240cba1562a, Normalized Score: 2.32%
Project ID: 0x2f6cbef6708256634282

In [ ]:
for category, scores in normalized_project_scores.items():
  for project_id, normalized_score in scores.items():
    score_total = normalized_category_medians[category]
    print(f"Project ID: {project_id}, Normalized Score: {(normalized_score/100)*(score_total/100)}")

Project ID: 0x82ef80ecf81b0bab897fe9ab595544db44b14708993f91c5f5f457f869d2af1d, Normalized Score: 0.008882563115598097
Project ID: 0xbdc75c7f2d9d4f797174656b47ef920cc968b82a1235efec4a37816fc8077ef9, Normalized Score: 0.005615994766998051
Project ID: 0xdda0798a5eb762366cd5aa5de87f042567946b47f5cf1d776077a2b00d75c575, Normalized Score: 0.007098969113594605
Project ID: 0xd7f9140cd24d5332f81c9a12d5d7da80f0c61f69db015073727064f0b975c875, Normalized Score: 0.006898768218020978
Project ID: 0xf615a763fa6599d40e52ee2e15ec77f208324c5863a157cdeb243639936476e8, Normalized Score: 0.005238849117243994
Project ID: 0x73deb2a75792f7c688dc87a3080ce18637b7d241ad2b551272fb21aee72d6bf7, Normalized Score: 0.0036941200367676006
Project ID: 0x762c363ac739a057eb4d414fd17916a2b098a00c7ddeae32d021f6df572561e1, Normalized Score: 0.006154673176596913
Project ID: 0x56cb9a566631cf918d682a40877cb4ffe7dfc4d3cbec30f5cc9a1a71c8083618, Normalized Score: 0.005783657101423993
Project ID: 0x0af93b34c4c047485dca950b6a1f06372

In [ ]:
### Calculate Results (weighted proportional distribution)

# 1. Multiply Project Scores with category scores - done
# 2. Adjust scores to match 100% (there should be no need to readjust) - done
# 3. Implement Max scores and redistribute excess
# 4. Implement min and redistribute excess (while not breaking max rule)

# Constants
total_amount = budget_median
max_amount = total_amount*0.125
# max_amount = 1000000
min_amount = 1000

# Step 1: Calculate total allocation for each category
category_total_allocations = {category: (allocation / 100) * total_amount for category, allocation in normalized_category_medians.items()}

# Final allocations dictionary
final_allocations = {}

# Step 2: Sort projects within each category
for category, scores in normalized_project_scores.items():
    normalized_project_scores[category] = dict(sorted(scores.items(), key=lambda x: x[1], reverse=True))

# Step 3: Calculate project allocations using weighted proportional distribution
for category, scores in normalized_project_scores.items():
    total_allocation = category_total_allocations[category]
    project_allocations = {}

    # Calculate total score for the category
    total_score = sum(scores.values())

    # Calculate weighted proportional allocations
    for project, score in scores.items():
        allocation = (score / total_score) * total_allocation
        project_allocations[project] = allocation

    # Step 4: Remove projects below min_amount
    project_allocations = {project: allocation for project, allocation in project_allocations.items() if allocation >= min_amount}

    # Step 5: Normalize allocations
    total_allocated = sum(project_allocations.values())

    if total_allocated > 0:
        normalized_allocations = {project: (allocation / total_allocated) * total_allocation for project, allocation in project_allocations.items()}
    else:
        normalized_allocations = {}

    # Step 6: Cap allocations at max_amount
    normalized_allocations = {project: min(allocation, max_amount) for project, allocation in normalized_allocations.items()}

    # Store the final allocations for the category
    final_allocations[category] = normalized_allocations

# Display the final allocations
for category, allocations in final_allocations.items():
    print(f"Allocations for {category}:")
    for project, allocation in allocations.items():
        print(f"  Project {project}: {allocation:,.2f}")


Allocations for eth_core:
  Project 0xf2021d8979bd4aa2227f27cfe6490815952fca725e8da6271e1f6dba49ba0701: 79,676.29
  Project 0x539cf7777f9ca3ec7a4ed2983c3248c36f3691cb004eddce3e0d377d0ef73375: 75,832.34
  Project 0x86f20a2671ef751e9fdf181bfe8d2c6e0647f67679489b197c26cb25dad5458f: 72,895.49
  Project 0xa958c93582454042b966df568c686264553dedd4262f890f98e1443123277eae: 67,646.93
  Project 0xa890fad98dd1b5179eef8c2d3a4ac52a51327d1ef51f826d5f583e725880af89: 66,800.67
  Project 0xc0dac906167f299014aeb6ad6d31d5194c2c73c7fe73405b5b9bef7d172a1487: 65,777.17
  Project 0xcdebeb77a8d4ebbf16aff97d72dcea3d080ae73e7848d858c2f051eb2d2aed2c: 65,509.88
  Project 0x82ef80ecf81b0bab897fe9ab595544db44b14708993f91c5f5f457f869d2af1d: 64,176.52
  Project 0x383672f18d21daf99befe6f41d201f357f70f1883daba5260ac992413edd1ba2: 63,804.41
  Project 0x6aeb3dad89f6557ec81a11d4c59afd43afda97e8358145eba8a701462f1923d2: 63,537.83
  Project 0x6c97a1517ef2f72228131c288c1a9256c7e4d47bba01d069b16df2f7ca81bd85: 62,949.50
  Proj